# Social_Media_Sentiment_Analysis 

In this NLP Project we aim to train a model to classify tweets from twitter to be "negative"(0) or "positive"(4)


In [1]:
#Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#Reading the data
#The dataset is taken from Kaggle (sentiment140 dataset)
df = pd.read_csv("new_twitter_data.csv",index_col = 0 )

C:\Users\Naushin\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,target,text
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1599999 entries, 1 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   text    1599999 non-null  object
dtypes: int64(1), object(1)
memory usage: 36.6+ MB


## Data Pre-processing

Given below are some functions which will be used to pre-process the data

In [5]:
#importing the necessary libraries
import nltk
import string
import re

In [6]:
# A dictionary containing some of the common emoticons
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

def replace_emoticon(text):
    """
    This function replaces emoticons with their equivalent emotion
    """
    for emoji in emojis.keys():
        text = text.replace(emoji, " EMOJI" + emojis[emoji] + " ") 
    return text
    

In [7]:
punc = string.punctuation
punc = punc.replace("&","")

def remove_punctuation(text):
    """
    This function removes all the punctuations included in the string punc with a blank space
    """
    s =[]
    for char in text:
        if char in punc:
            s.append(" ")
        else:
            s.append(char)           
    return "".join(s)

In [8]:
def remove_ampersands(text):
    """
    When the dataset is scrapped from the some unecessary ampersanded words can arise which are removed by this function
    """
    words = text.split();
    n = len(words)
    for i in range(0,n):
        if "&" in words[i]:
            temp = words[i].split("&")
            if len(temp) == 1:
                words[i]  =""
            else:
                words[i] = temp[0]
    return " ".join(words)            

In [9]:
def remove_numbers(text):
    """
    This function removes numbers
    """
    result = re.sub(r'\d+', '', text)
    return result

In [10]:
def util_func(text):
    n = len(text)
    flag  = 0;
    char = ""
    s = []
    for i in range(0,n):
        if char!=text[i]:
            flag = 1
            char = text[i]
            s.append(char)
            continue            
        if char==text[i] and flag==1:
            flag = 2
            s.append(char)
            continue            
        if flag==2 and char==text[i]:
            continue            
    return "".join(s)       


def remove_morethan2letters(text):
    """
    if more than 2 letters occur consecutively they are replaced by a 2 letters
    """
    words = text.split();
    n = len(words)
    for i in range(0,n):
        words[i] = util_func(words[i])
        
    return " ".join(words) 
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Naushin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

def lemmatize_word(text):
    """
    Lemmatizes the text
    """
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in tokens]
    lemmas = ' '.join(lemmas)
    return lemmas

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Naushin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
#Stopwords
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove("nor")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def remove_stopwords(text):
    """
    Removes stopwords from the text
    """
    tokens = word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words]
    filtered = ' '.join(filtered)
    return filtered

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Naushin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#the analyser function which includes all the aboove functions
def text_preprocessor(text):

    text = text.lower()
    text = replace_emoticon(text)    
    text = remove_punctuation(text) 
    text = remove_ampersands(text)
    text = remove_numbers(text)
    text = remove_morethan2letters(text)
    text = lemmatize_word(text)
    text = remove_stopwords(text)
    
    return text


## Training the model


In [15]:
#importing the necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [16]:
#creating pipeline model to vectorize text and fit in the model
model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), LogisticRegression(max_iter=500))

In [17]:
#preprocessing the text
X = df["text"].apply(text_preprocessor)

#splitting the data 
X_train, X_test, y_train, y_test =train_test_split(X,df["target"], test_size=0.1)


In [18]:
#Fitting the model with training data
model.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercep

## Testing the Model

In [19]:
#Making predictions on the test data
predictions = model.predict(X_test)

In [20]:
#Printing the classification report
from sklearn.metrics import classification_report
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80     79363
           4       0.80      0.80      0.80     80637

    accuracy                           0.80    160000
   macro avg       0.80      0.80      0.80    160000
weighted avg       0.80      0.80      0.80    160000



The accuracy of the model is 80%